## Import

In [26]:
import dagsim.base as ds
import numpy as np
from PIL import Image
import shutil

In [27]:
import os
print(os.getcwd())

/home/ghadi/Documents/PhD/dagsimManuscript


In [105]:
def random_exclude(range_start, range_end, excludes):
    r = random.randint(range_start, range_end)
    if r in excludes:
        return random_exclude(range_start, range_end, excludes)
    return r

def Ber(U1):
    return np.random.binomial(1, 1-U1)

def BerExp(C, Dnum, Dstr):
    if Dstr=="H":
        out = 0.75*Dnum + 0.5*C + 0.25
    else:
        out = 2.5*Dnum + 1.75*C - 0.25
    out = 1/(1 + np.exp(-out))
    out = np.random.binomial(1, out)
    return out

def drawImage(H, V, R, C, output_path):
    image = np.zeros(shape=(256, 256))
    randInd = np.random.randint(low=1, high=10000)
    if H==1:
        randPosH = np.random.randint(low=10, high=246)
        print(randPosH)
        image[randPosH-5:randPosH+5, :]=256
        
    if V==1:
        randPosV = np.random.randint(low=10, high=246)
        print(randPosV)
        image[:, randPosV-5:randPosV+5]=256
        
    if C==1:
        randC1 = np.random.randint(low=10, high=246)
        randC2 = np.random.randint(low=10, high=246)

        circle = (randC1) ** 2 + (randC2) ** 2
        donut = numpy.logical_and(circle < (5), circle > (5))
        
        
    image = image + np.random.binomial(1, 0.005, size=(256,256))*256
    image = Image.fromarray(image)
    image = image.convert("L")
    image.save(output_path + "/" + str(randInd) + ".png")

In [106]:
U1 = ds.Generic(name="U_1", function=np.random.uniform)
U2 = ds.Generic(name="U_2", function=np.random.uniform)

H = ds.Generic(name="H", function=np.random.binomial, arguments={"n":1, "p":U1})
C = ds.Generic(name="C", function=np.random.binomial, arguments={"n":1, "p":U2})

V = ds.Generic(name="V", function=Ber, arguments={"U1":U1})

R = ds.Generic(name="R", function=BerExp, arguments={"C":C, "Dnum":H, "Dstr":"H"})
Y = ds.Generic(name="Y", function=BerExp, arguments={"C":C, "Dnum":V, "Dstr":"V"})

I = ds.Generic(name="Image", function=drawImage, arguments={"H":H, "V":V, "R":R, "C":C}, visible=False)

graph = ds.Graph(name="testImages", list_nodes=[U1, U2, H, V, C, R, Y, I])
# graph.draw()

In [107]:
if os.path.exists('./images'):
    shutil.rmtree('./images')
os.mkdir("./images")

n = graph.simulate(num_samples=10, output_path="./images")

Simulation started
76
46
test  46
191
test  191
181
test  181
93
208
test  208
175
test  175
92
test  92
Simulation finished in 0.0306 seconds


In [98]:
np.random.binomial(1, 0.3, size=(10,10))

array([[0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0]])

In [110]:
Y, X = np.ogrid[:10, :10]
center = [5,5]
print(X)
print(Y)

[[0 1 2 3 4 5 6 7 8 9]]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]


In [111]:
dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

In [112]:
dist_from_center

array([[7.07106781, 6.40312424, 5.83095189, 5.38516481, 5.09901951,
        5.        , 5.09901951, 5.38516481, 5.83095189, 6.40312424],
       [6.40312424, 5.65685425, 5.        , 4.47213595, 4.12310563,
        4.        , 4.12310563, 4.47213595, 5.        , 5.65685425],
       [5.83095189, 5.        , 4.24264069, 3.60555128, 3.16227766,
        3.        , 3.16227766, 3.60555128, 4.24264069, 5.        ],
       [5.38516481, 4.47213595, 3.60555128, 2.82842712, 2.23606798,
        2.        , 2.23606798, 2.82842712, 3.60555128, 4.47213595],
       [5.09901951, 4.12310563, 3.16227766, 2.23606798, 1.41421356,
        1.        , 1.41421356, 2.23606798, 3.16227766, 4.12310563],
       [5.        , 4.        , 3.        , 2.        , 1.        ,
        0.        , 1.        , 2.        , 3.        , 4.        ],
       [5.09901951, 4.12310563, 3.16227766, 2.23606798, 1.41421356,
        1.        , 1.41421356, 2.23606798, 3.16227766, 4.12310563],
       [5.38516481, 4.47213595, 3.6055512

In [ ]:
mask = dist_from_center <= radius